In [1]:
import yfinance as yf

ticker = "AAPL"
prices = yf.Ticker(ticker).history(period="1y")

prices['high'] = prices['High']
prices['low'] = prices['Low']
prices['close'] = prices['Close']
prices['open'] = prices['Open']

In [2]:
from mplchart.zigzag import Zigzag

zigzag = Zigzag(backcandels=5, forwardcandels=5, pivot_limit=50, offset=0)
zigzag.calculate(prices)

for pivot in zigzag.zigzag_pivots:
    print(f"Pivot: time={pivot.point.time}, index={pivot.point.index}, price={pivot.point.price}, norm_price={pivot.point.norm_price}")

Pivot: time=2024-11-08 00:00:00-05:00, index=249, price=228.66000366210938, norm_price=0.9048834035369343
Pivot: time=2024-11-04 00:00:00-05:00, index=245, price=219.46854635353304, norm_price=0.8028574523372206
Pivot: time=2024-10-15 00:00:00-04:00, index=231, price=237.22900130875718, norm_price=1.0
Pivot: time=2024-10-07 00:00:00-04:00, index=225, price=221.08676038639473, norm_price=0.8208197645587426
Pivot: time=2024-09-20 00:00:00-04:00, index=214, price=232.83382873595048, norm_price=0.9512132138165854
Pivot: time=2024-09-16 00:00:00-04:00, index=210, price=213.6849038037586, norm_price=0.7386585323259963
Pivot: time=2024-08-29 00:00:00-04:00, index=199, price=232.66402541964027, norm_price=0.9493283826867873
Pivot: time=2024-08-05 00:00:00-04:00, index=181, price=195.55824424838744, norm_price=0.5374510827211876
Pivot: time=2024-08-02 00:00:00-04:00, index=180, price=225.09153987705304, norm_price=0.8652731546102062
Pivot: time=2024-07-25 00:00:00-04:00, index=174, price=214.13

In [3]:
from mplchart.primitives import ScanProperties
from mplchart.trendlines import find_pattern, TrendLine
from typing import List

scan_props = ScanProperties(offset=0, number_of_pivots=5, error_ratio=0.02)

# Initialize pattern storage
patterns: List[TrendLine] = []

# Find patterns
for i in range(0, len(zigzag.zigzag_pivots)):
    find_pattern(zigzag, i, scan_props, patterns, prices)

for pattern in patterns:
    print(f"{pattern.pattern_name}: {pattern.pivots[0].point.index} - {pattern.pivots[-1].point.index}")


Points: 214, 231, 249 on the same line, cos_diff=3.324374922153428e-05
Chose points: 214, 231
Points: 210, 225, 245 on the same line, cos_diff=2.0323278750677964e-05
Chose points: 210, 245
Points: 199, 214, 231 on the same line, cos_diff=3.765170842440213e-06
Chose points: 199, 231
Points: 181, 210, 225 on the same line, cos_diff=1.0668452573581177e-06
Chose points: 210, 225
Points: 180, 199, 214 on the same line, cos_diff=9.237570211428903e-06
Chose points: 180, 214
Points: 174, 181, 210 on the same line, cos_diff=0.000662015020259088
Chose points: 174, 181
Points: 166, 180, 199 on the same line, cos_diff=9.280354467133822e-05
Chose points: 166, 199
Points: 152, 174, 181 on the same line, cos_diff=0.0005608559040981564
Chose points: 152, 181
Points: 145, 166, 180 on the same line, cos_diff=0.00016527074247196794
Chose points: 145, 166
Points: 132, 152, 174 on the same line, cos_diff=2.457055684990106e-05
Points: 118, 145, 166 on the same line, cos_diff=1.1234583027430745e-05
Chose poi